In [1]:
import argparse
import torch
import matplotlib.pyplot as plt

from torchvision import transforms
import cv2

from retinanet import model
from retinanet.dataloader import CocoDataset, CSVDataset, collater, Resizer, AspectRatioBasedSampler, Augmenter, \
	UnNormalizer, Normalizer
from retinanet import csv_eval
from retinanet.losses import *
# Inmport superpoint
from retinanet.networks.superpoint_pytorch import SuperPointFrontend

In [2]:
csv_annotations_path = '/home/baudoin/pytorch-retinanet-pipeline/visdrone_anno/annotations_dev.csv'
class_list_path = '/home/baudoin/pytorch-retinanet-pipeline/visdrone_anno/class_labels.csv'
model_path = '/home/baudoin/pytorch-retinanet-pipeline/results/training_results/20220113-191006/checkpoints/checkpoint_csv_retinanet_16.pt'
iou_threshold = 0.5

In [8]:
dataset_val = CSVDataset(csv_annotations_path,class_list_path,transform=transforms.Compose([Normalizer(), Resizer()]))

superpoint = SuperPointFrontend(project_root='') #Modify project_root

 Load SuperPoint Pth from  checkpoints/superpoint/superpoint_v1.pth


In [4]:
def layer_to_img(layer):
    layer = layer.squeeze()[1:].unsqueeze(0)
    pixel_shuffle = torch.nn.PixelShuffle(8)
    output = pixel_shuffle(layer)
    print('img shape')
    print(output.size())

    return output.squeeze()

def draw_keypoints(img, corners, color):
    keypoints = [cv2.KeyPoint(c[1], c[0], 1) for c in np.stack(corners).T]
    return cv2.drawKeypoints(img.astype(np.uint8), keypoints, None, color=color)

In [7]:
dataset = dataset_val
del dataset_val
iou_threshold=0.5
score_threshold=0.05
max_detections=100
save_path=None

for index in range(len(dataset)):
    data = dataset[index]
    print(data.keys())
    scale = data['scale']

    

    # SuperPoint label with teacher model
    output_superpoint = superpoint.run(data['img_gray'].permute(2, 0, 1).cuda().float().unsqueeze(dim=0))
    desc_teacher = torch.from_numpy(output_superpoint['local_descriptor_map']).type(torch.FloatTensor)#.to(device)
    dect_teacher = torch.from_numpy(output_superpoint['dense_scores']).type(torch.FloatTensor)#.to(device)
    
    img_original = data['img_gray'].numpy()*255
    keypoints = layer_to_img(F.softmax(dect_teacher, dim=1))
    corners = np.array(np.where(keypoints > 0.9)).astype('float')
    img = draw_keypoints(img_original, corners, (0,255,0))
    
    plt.figure(figsize=(18,16))
    plt.imshow(img)
    plt.show()
    plt.figure(figsize=(18,16))
    plt.imshow(img_original)
    plt.show()


    
        

        


NameError: name 'dataset_val' is not defined

In [ ]:
            
            
            print(np.shape((np.array(output_superpoint['local_descriptor_map']))))
            print(np.shape((np.array(output_superpoint['dense_scores']))))
            import matplotlib.pyplot as plt
            import cv2
            def draw_keypoints(img, corners, color):
                keypoints = [cv2.KeyPoint(c[1], c[0], 1) for c in np.stack(corners).T]
                return cv2.drawKeypoints(img.astype(np.uint8), keypoints, None, color=color)
            def plot(img, fig = 0):
                plt.figure(fig)
                plt.imshow(img, cmap='gray', vmin=0, vmax=1)
                plt.show()  # display it
                plt.savefig('/home/baudoin/pytorch-retinanet-pipeline/img1.png')
            def layer_to_img(layer):
                layer = layer.squeeze()[1:].unsqueeze(0)
                pixel_shuffle = torch.nn.PixelShuffle(8)
                output = pixel_shuffle(layer)
                print('img shape')
                print(output.size())

                return output.squeeze()
            keypoints = layer_to_img(F.softmax(output_semi, dim=1))
            
            print(data['img_gray'])

            frame = draw_keypoints(data['img_gray'], np.where(keypoints > 0.5), (0,255,0))
